In [ ]:
import pandas as pd
import numpy as np
import sqlalchemy
from datetime import datetime
import time
import ta

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from data.get_data import *

from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

In [ ]:
from data import db


SYMBOL = "BTC"
start = "2020-01"
end = "2024"

DB = db.connect_db("database", interval="1h")
data = DB.get_data("BTC")
data = data.loc[start : end]


In [ ]:
data = data[['close', 'volume']].rename(columns={'close' : 'price'}).copy()
data

## Barres de temps

In [ ]:
data['volume'].describe()

In [ ]:
# Barres de Volume
def volume_bars(data, volume_size=100):
    data['cum_volume'] = data['volume'].cumsum()
    data['volume_bin'] = data['cum_volume'] // volume_size
    return data.groupby('volume_bin').agg({
        'price': ['first', 'last', 'max', 'min', 'mean'],
        'volume': 'sum'
    })
    
    
volume_bars(data, 2278)

In [ ]:
data.shape

In [ ]:

import matplotlib.pyplot as plt

# Barres de Ticks
def tick_bars(data, tick_size=100):
    data['tick_count'] = np.arange(len(data)) // tick_size
    return data.groupby('tick_count').agg({
        'price': ['first', 'last', 'max', 'min', 'mean'],
        'volume': 'sum'
    })

# Barres de Volume Adaptatif
def adaptive_volume_bars(data, base_volume_size, volatility_threshold=0.01):
    data['volatility'] = data['price'].rolling(window=24).std().fillna(0)
    data['adaptive_volume_size'] = base_volume_size * (1 + data['volatility'] / volatility_threshold)
    
    bars = []
    cum_volume = 0
    volume_bin_start = data.index[0]
    for index, row in data.iterrows():
        cum_volume += row['volume']
        if cum_volume >= row['adaptive_volume_size']:
            bars.append({
                'start_time': volume_bin_start,
                'end_time': index,
                'open': data.loc[volume_bin_start]['price'],
                'close': row['price'],
                'high': data.loc[volume_bin_start:index]['price'].max(),
                'low': data.loc[volume_bin_start:index]['price'].min(),
                'mean': data.loc[volume_bin_start:index]['price'].mean(),
                'volume': cum_volume
            })
            volume_bin_start = index
            cum_volume = 0
    return pd.DataFrame(bars)

# Appliquer les méthodes de construction de barres
tick_bar_result = tick_bars(df)
adaptive_volume_bar_result = adaptive_volume_bars(df, base_volume_size=100)

# Convertir les résultats pour correspondre aux mêmes colonnes
tick_bar_result.columns = ['open', 'close', 'high', 'low', 'mean', 'volume']
adaptive_volume_bar_result.set_index('end_time', inplace=True)


In [ ]:
px.line(adaptive_volume_bar_result['close'])

In [ ]:
fig = px.line(data['close'])
fig.show()

In [ ]:
adaptive_volume_bar_result

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x = adaptive_volume_bar_result.index, y = adaptive_volume_bar_result['close']
    )
)

fig.add_trace(
    go.Scatter(
        x = data.index, y = data['close']
    )
)


fig.update_xaxes(rangeslider_visible = False)

fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)
fig

In [ ]:
fig = px.line(y=data['close'])
fig.show()

In [ ]:
adaptive_volume_bar_result

In [ ]:
fig = go.Figure()
#shared_xaxes=True, row_heights=[0.6, 0.2, 0.2]
fig.add_trace(
    go.Candlestick(
        x = data.index , open = data.open, close = data.close,
        high = data.high, low = data.low, name = SYMBOL
    )
)

fig.update_xaxes(rangeslider_visible = False)
fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)

fig.show()

In [ ]:
g g g

# Etiquette

TypeError: Feature names are only supported if all input features have string names, but your input has ['quoted_name', 'str'] as feature name / column name types. If you want feature names to be stored and validated, you must convert them all to strings, by using X.columns = X.columns.astype(str) for example. Otherwise you can remove feature / column names from your input data, or convert them all to a non-string data type.